# **HitlerGPT - inference**
https://github.com/FENRlR/HitlerGPT

In [ ]:
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install datasets

In [ ]:
!git clone https://github.com/FENRlR/HitlerGPT
%cd HitlerGPT
!ls

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/pythia-410m-deduped"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token # needed for gpt-neo-x tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

In [ ]:
outfolder = "model2"
resumep = f"{outfolder}"
from peft import PeftModel
model = PeftModel.from_pretrained(model, resumep)

In [ ]:
def gen3(x):
    gened = model.generate(
        **tokenizer(
            x,
            return_tensors='pt',
            return_token_type_ids=False,
        ).to(0),
        max_new_tokens=256,
        early_stopping=True,
        do_sample= True,
        eos_token_id=2,
        repetition_penalty=1.2,
    )
    return tokenizer.decode(gened[0]).replace('#','')

In [ ]:
while 1:
  q = input('user > ').strip()
  if q == 'quit':
    break
  print(gen3(q))